In [21]:
import json
import numpy as np
import gensim
import pandas as pd
from scipy.spatial.distance import cosine

In [22]:
def cosine_sim(v,w):
    return 1.0 - cosine(v,w)

### 必須課題
> コーパスを選択し、ベクトル空間モデルに基づいた検索を実現せよ.  
> ３種類以上のクエリでの検索結果を示すこと.

#### おさらい
+ コーパスとは「文書集合」のこと
+ 辞書とは「単語」と「単語のID」との対応付けのこと
+ TF-IDFはコーパス内の各文書での単語の出現頻度に基づいた尺度の一つ

#### 基本方針
1. コーパスの読み込み
2. 単語→単語IDの辞書を作成する
  + 精度が芳しくなければ、日本語のストップワードリストを利用する
3. 単語IDごとに出現回数をカウントする
4. 出現頻度に応じてTF-IDFで重み付けされた特徴ベクトルを作成する
5. 各文書のベクトル表現を作成する
6. クエリをベクトル表現に変換し、類似度の高いものを取得する

In [5]:
# [START コーパスの読み込み]
with open("../data/kyoto_results_100.json", "r") as f:
    docs = json.load(f)
print("# of docs: {}".format(len(docs)))
# [END コーパスの読み込み]

# of docs: 83


In [9]:
# [START 単語→単語IDの辞書を作成する]
assert type(docs[-1]["bow"]) == str
dictionary = gensim.corpora.Dictionary([doc["bow"].split() for doc in docs])
for idx, (key, val) in enumerate(dictionary.token2id.items()):
    if idx < 3:
        print(key, val)
# [END 単語→単語IDの辞書を作成する]

定番 0
穴場 1
お 2


辞書には「お」というように、意味をなさないような単語も含まれている.  
検索の精度を向上させるためには、このような無意味な単語を除去する必要がありそうだ.

In [13]:
# [START 単語IDごとに出現回数をカウントする]
id_corpus = [dictionary.doc2bow(doc["bow"].split()) for doc in docs]
# [END 単語IDごとに出現回数をカウントする]

In [14]:
# [START 出現頻度に応じてTF-IDFで重み付けされた特徴ベクトルを作成する]
tfidf_model = gensim.models.TfidfModel(id_corpus, normalize=False)
tfidf_corpus = tfidf_model[id_corpus]
# [END 出現頻度に応じてTF-IDFで重み付けされた特徴ベクトルを作成する]

In [17]:
# [START 各文書のベクトル表現を作成する]
tfidf_vectors = gensim.matutils.corpus2dense(tfidf_corpus, len(dictionary)).T
print(tfidf_vectors[0])
print(np.asarray(tfidf_vectors).shape)
# [END 各文書のベクトル表現を作成する]

[ 7.13536882  6.75007868  3.96544409 ...,  0.          0.          0.        ]
(83, 1160)


各文書について、1160次元のベクトルとして表現されていることが確認できる.

In [46]:
class BasicEngine(object):
    def __init__(self, docs):
        """辞書の作成
        """
        self.dictionary = gensim.corpora.Dictionary([doc["bow"].split() for doc in docs])
        self.dict_size = len(dictionary)
        
    def _calc_cossim(self, v, w):
        return 1.0 - cosine(v, w)
        
class TfidfEngine(BasicEngine):
    """TF-IDFでの文書検索の実装
    """
    def __init__(self, docs):
        super().__init__(docs)
        id_corpus = [self.dictionary.doc2bow(doc["bow"].split()) for doc in docs]
        self.tfidf_model = gensim.models.TfidfModel(id_corpus, normalize=False)
        self.tfidf_corpus = tfidf_model[id_corpus]
        self.tfidf_vectors = gensim.matutils.corpus2dense(self.tfidf_corpus, len(self.dictionary)).T

    def search(self, query, n_results = 5):
        assert type(query) == set, "クエリはdict型で渡す"
        scores = self._calc_scores(query)
        s = pd.Series(scores)
        return [(idx, score) for idx, score in s.sort_values(ascending=False)[:n_results].iteritems()]

    def _calc_scores(self, query):
        """クエリと文書のスコア計算
        
        returns
        ---------
        scores : array, 各文書のスコア
        """
        tfidf_q = tfidf_model[self.dictionary.doc2bow(query)]
        query_vec = gensim.matutils.corpus2dense([tfidf_q], self.dict_size).T[0]
        return [self._calc_cossim(query_vec, doc_vec) for doc_vec in self.tfidf_vectors]

In [50]:
# [START 3種類のクエリで検索結果を示す]
tfidf_engine = TfidfEngine(docs)
results = tfidf_engine.search({"観光"})
for idx, score in results:
    print(docs[idx]["title"])
# [(dictionary[x[0]], x[1]) for x in tfidf_corpus[0]]
# + コーパス中の文書についてクエリとの類似度を計算する
# + 文書番号と類似度の対応表を作る
# + 類似度の分布みる ひょっとしたらベクトルの作り方やクエリによって分布に偏りがあるかも
# [END 3種類のクエリで検索結果を示す]

旬の京都観光情報と 京都のおすすめ観光スポット | 京都観光総 ...
観光バスツアー・観光タクシー | 京都の観光&遊び・体験 ...
「 京都観光タクシー 」 格安で有意義な京都観光をお約束！
京都観光おすすめのグルメや人気の穴場
京都観光/旅行・ぶらり伏見
